# Maximum Likelihood (ML) and Maximum a Posteriori (MAP)

## implementation of necessary functions

In [1]:
import numpy as np
import scipy
import scipy.linalg
import matplotlib.pyplot as plt

np.random.seed(14)

# The gradient_descent implementation.
def gradient_descent(f, grad_f, X_tilda, w_init, y_noisy, kmax, tolf, tolx):
    
    # Initialization
    w = np.zeros((kmax, w_init.size))
    f_val = np.zeros((kmax, 1))
    grads = np.zeros((kmax, w_init.size))
    err = np.zeros((kmax, 1))
    
    # Assign the values for the first iteration
    w[0, :] = w_init
    f_val[0] = f(X_tilda, w[0, :], y_noisy)
    grads[0, :] = grad_f(X_tilda, w[0, :], y_noisy)
    err[0] = np.linalg.norm(grads[0, :], ord=2)
    
    # Choose step size
    alpha = 0.1
    
    # Handle the condition for the first iteration
    # if k = 0 then w[k-1, :] = w[-1, :]
    w[-1, :] = np.ones((w_init.size,))
    
    # Start the iterations
    k = 0
    while (k < kmax-1 and
          np.linalg.norm(grads[k, :], ord=2) > tolf * np.linalg.norm(grads[k-1, :], ord=2) and
          np.linalg.norm((w[k, :]-w[k-1, :]), ord=2) > tolx):
        
        k = k + 1
        
        # Update the step size alpha
        alpha = backtracking(f, grad_f, X_tilda, w[k, :], y_noisy)
        
        # Update the value of x
        w[k, :] = w[k-1, :] - alpha * grads[k-1, :]
        
        # Update the values the the actual iteration
        f_val[k] = f(X_tilda, w[k, :], y_noisy)
        grads[k, :] = grad_f(X_tilda, w[k, :], y_noisy)
        err[k] = np.linalg.norm(grads[k, :], ord=2)
        
        
    
    # Truncate the vectors that are (eventually) too long
    f_val = f_val[:k+1]
    grads = grads[:k+1,:]
    err = err[:k+1]
    w = w[:k+1,:]
    return w, k, f_val, grads, err

# modified backtracking
def backtracking(f, grad_f, X_tilda, w, y_noisy):
    """
    This function is a simple implementation of the backtracking algorithm for
    the GD (Gradient Descent) method.
    
    f: function. The function that we want to optimize.
    grad_f: function. The gradient of f(x).
    x: ndarray. The actual iterate x_k.
    """
    alpha = 1
    c = 0.8
    tau = 0.25
    
    while f(X_tilda, (w - alpha * grad_f(X_tilda, w, y_noisy)), y_noisy) > f(X_tilda, w, y_noisy) - c * alpha * np.linalg.norm(grad_f(X_tilda, w, y_noisy), 2) ** 2:
        alpha = tau * alpha
        # next line prevent alpha to go extremely low
        if alpha < 1e-5:
            break
    return alpha

## creation of dataset

In [2]:
# create a N x n+1 Vandermode matrix
n = 3   # the true degree of polynomial
N =  20 # number of samples

# input
x = np.linspace(0, 1, N)

# vandermonde creation
X = np.vander(x, n)

# true weights
w_true = np.ones((n,))

# outputs
y_true = X @ w_true

# add normal noise (mean=0, std=0.1)
y_noisy = y_true + np.random.normal(0, 0.1, y_true.shape)